In [3]:
import steamreviews
import requests
import pandas as pd
import numpy as np
import os

request_params = dict()
request_params['language'] = 'english'
#request_params['day_range'] = '28'

# Elden Ring appID: 1245620
# Ember Knights appID: 1135230
app_id = [1135230]
#review_dict = steamreviews.download_reviews_for_app_id_batch(app_id, chosen_request_params=request_params)
#review_dict = steamreviews.download_reviews_for_app_id_batch(app_id)

In [4]:
# Steam Reviews Dataset - https://www.kaggle.com/datasets/luthfim/steam-reviews-dataset
df = pd.read_csv('data/steam_reviews.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434891 entries, 0 to 434890
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   date_posted             434891 non-null  object
 1   funny                   434891 non-null  int64 
 2   helpful                 434891 non-null  int64 
 3   hour_played             434891 non-null  int64 
 4   is_early_access_review  434891 non-null  bool  
 5   recommendation          434891 non-null  object
 6   review                  433375 non-null  object
 7   title                   434891 non-null  object
dtypes: bool(1), int64(3), object(4)
memory usage: 23.6+ MB


In [5]:
# Create a sample of reviews for creating test data

# Number of reviews
n = 500

# Only take reviews of minimum length
df_filtered = df[df['review'].str.len() >= 100]

# Only take reviews with a helpful score > 1
df_filtered = df[df['helpful'] > 1]

# Ensure a good distribution of different titles
title_distribution = df_filtered['title'].value_counts(normalize=True)

sampled_reviews = []
for title, proportion in title_distribution.items():
    n_samples = int(proportion * n)  # Number of samples per title
    title_reviews = df_filtered[df_filtered['title'] == title]
    sampled_reviews.append(title_reviews.sample(n=n_samples, replace=False))  #'replace'=False ensures no duplicates
final_sample = pd.concat(sampled_reviews)
final_sample = final_sample.sample(frac = 1)

In [6]:
final_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 483 entries, 219127 to 55535
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   date_posted             483 non-null    object
 1   funny                   483 non-null    int64 
 2   helpful                 483 non-null    int64 
 3   hour_played             483 non-null    int64 
 4   is_early_access_review  483 non-null    bool  
 5   recommendation          483 non-null    object
 6   review                  477 non-null    object
 7   title                   483 non-null    object
dtypes: bool(1), int64(3), object(4)
memory usage: 30.7+ KB


In [7]:
display(final_sample)
final_sample.to_csv('data/sample_reviews.csv')

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title
219127,2018-01-05,1,3,194,False,Not Recommended,ugly game bad game DONT BUY. DONT BUY DONT BUY...,PLAYERUNKNOWN'S BATTLEGROUNDS
245552,2018-02-10,1,3,1155,False,Not Recommended,Product received for free. This game is great ...,PLAYERUNKNOWN'S BATTLEGROUNDS
116496,2015-05-24,3,4,339,False,Recommended,Took a plane and crashed it on a road killing ...,Grand Theft Auto V
133561,2015-05-05,3,20,758,False,Not Recommended,I remember when a good fov slider was in games...,Grand Theft Auto V
265262,2017-08-11,1,2,165,True,Recommended,This game is amazing,PLAYERUNKNOWN'S BATTLEGROUNDS
...,...,...,...,...,...,...,...,...
364419,2015-10-20,1,2,724,True,Recommended,At school I'm a 14 year old boy but in Rust I'...,Rust
365648,2016-11-11,1,3,36,True,Not Recommended,Product received for free. Runs like hot garba...,Rust
391576,2015-06-26,1,3,217,True,Recommended,people judge me by my ingame size10/10,Rust
51902,2017-12-19,2,2,40,False,Not Recommended,"Got Banned for using ""Abussive/Offensive Chat/...",Rocket League®


In [8]:
# Number of unique titles
n = len(pd.unique(final_sample['title']))
print(n)

9


In [ ]:
!pip install datasets

In [9]:
from datasets import load_dataset
reviews_dataset = load_dataset('csv', data_files='annotated_reviews.csv', split="train[:50%]")

In [ ]:
## SetFitABSA test
!pip install "setfit[absa]"
!spacy download en_core_web_lg
from setfit import AbsaTrainer, AbsaModel

model = AbsaModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

trainer = AbsaTrainer(model, train_dataset=reviews_dataset)
trainer.train()

In [ ]:
model.save_pretrained(
    "models/setfit-absa-model-aspect", 
    "models/setfit-absa-model-polarity"
)